In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
import os
import numpy as np
import pandas as pd 
# import torch
# from random import randint
# from torchvision import transforms, models
# from torchvision.datasets import ImageFolder
# from torch.utils.data import DataLoader, random_split
# from torch import nn
# from torch import optim
# import time
# from fastai.vision import *
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator

In [3]:
metadata = pd.read_csv('/content/drive/My Drive/TEES/Trabalho/HAM10000_metadata')
metadata.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization,dataset
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp,vidir_modern
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp,vidir_modern
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp,vidir_modern
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp,vidir_modern
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear,vidir_modern


In [ ]:
# Gathering the test dataset images, fragmented for it's size, in a single folder.
# It's not necessary to run this cell again
old_path1 = '/content/drive/My Drive/TEES/Trabalho/HAM10000_part_2/'
new_path1 = '/content/drive/My Drive/TEES/Trabalho/HAM10000_part_1/'
archives = os.listdir(old_path1)
for i in archives:
  os.replace((old_path1+i), (new_path1+i))

In [ ]:
# Number of images in the folders
print(len(os.listdir(old_path1)))
print(len(os.listdir(new_path1))) 

0
10015


In [ ]:
# Organizing the images in folders representing their classes
# It's not necessary to run this cell again

old_path_2 = '/content/drive/My Drive/TEES/Trabalho/HAM10000_part_1/'
new_path_2 = '/content/drive/My Drive/TEES/Trabalho/dataset/'

for i in range(0, len(metadata)):
  os.replace(old_path_2+metadata.image_id[i]+'.jpg', new_path_2+metadata.dx[i]+"/"+metadata.image_id[i]+'.jpg')

In [ ]:
print(len(os.listdir(old_path_2)))
print(len(os.listdir(new_path_2+'akiec')))
print(len(os.listdir(new_path_2+'bcc')))
print(len(os.listdir(new_path_2+'bkl')))
print(len(os.listdir(new_path_2+'df')))
print(len(os.listdir(new_path_2+'mel')))
print(len(os.listdir(new_path_2+'nv')))
print(len(os.listdir(new_path_2+'vasc')))

0
327
514
1099
115
1113
6721
142


In [4]:
metadata_validation = pd.read_csv('/content/drive/My Drive/TEES/Trabalho/metadata_validation.csv')
metadata_validation.head()

,image,MEL,NV,BCC,AKIEC,BKL,DF,VASC
0,ISIC_0034321,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,ISIC_0034322,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,ISIC_0034323,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,ISIC_0034324,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,ISIC_0034325,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [32]:
# Now organizing the validation dataset
#It's not necessary to run this cell again
path3 = '/content/drive/My Drive/TEES/Trabalho/dataset_validation/'

for i in range(1, len(metadata_validation)):
  if metadata_validation.MEL[i] == 1.0:
    os.replace((path3+metadata_validation.image[i]+'.jpg'), path3+'mel/'+metadata_validation.image[i]+'.jpg')
  elif metadata_validation.NV[i] == 1.0:
    os.replace((path3+metadata_validation.image[i]+'.jpg'), path3+'nv/'+metadata_validation.image[i]+'.jpg')
  elif metadata_validation.BCC[i] == 1.0:
    os.replace((path3+metadata_validation.image[i]+'.jpg'), path3+'bcc/'+metadata_validation.image[i]+'.jpg')
  elif metadata_validation.AKIEC[i] == 1.0:
    os.replace((path3+metadata_validation.image[i]+'.jpg'), path3+'akiec/'+metadata_validation.image[i]+'.jpg')
  elif metadata_validation.BKL[i] == 1.0:
    os.replace((path3+metadata_validation.image[i]+'.jpg'), path3+'bkl/'+metadata_validation.image[i]+'.jpg')
  elif metadata_validation.DF[i] == 1.0:
    os.replace((path3+metadata_validation.image[i]+'.jpg'), path3+'df/'+metadata_validation.image[i]+'.jpg')
  elif metadata_validation.VASC[i] == 1.0:
    os.replace((path3+metadata_validation.image[i]+'.jpg'), path3+'vasc/'+metadata_validation.image[i]+'.jpg')

In [33]:
print(len(os.listdir(path3+'akiec')))
print(len(os.listdir(path3+'bcc')))
print(len(os.listdir(path3+'bcc')))
print(len(os.listdir(path3+'df')))
print(len(os.listdir(path3+'mel')))
print(len(os.listdir(path3+'nv')))
print(len(os.listdir(path3+'vasc')))

8
15
22
1
22
122
3


In [ ]:
# def testar_gpu():
# 	train_on_gpu = torch.cuda.is_available() #Observa se a GPU está disponivel
# 	if train_on_gpu: #Se sim
# 		device = torch.device('cuda') #Seleciona o device como GPU
# 		print("Treinando na GPU") #E manda a mensagem
# 	else: #Se não
# 		device = torch.device('cpu') #Seleciona o device como cpu
# 		print("GPU indisponível, treinando na CPU") #E avisa que a GPU não esta disponível
# 	return device

# device = testar_gpu()

Treinando na GPU


In [ ]:
# transform = transforms.Compose([
#                                 transforms.RandomHorizontalFlip(),
#                                 transforms.RandomRotation(10),
#                                 transforms.Resize((255, 255)),
#                                 transforms.ToTensor(),
#                                 transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
#                               ])
# data = ImageFolder('/content/drive/My Drive/TEES/Trabalho/dataset/', transform=transform)
# print('Total de imagens no dataset: ', len(data))

Total de imagens no dataset:  10015


In [5]:
datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
data = datagen.flow_from_directory('/content/drive/My Drive/TEES/Trabalho/dataset/',
                                                 target_size = (255, 255),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

data_val = datagen.flow_from_directory('/content/drive/My Drive/TEES/Trabalho/dataset_validation/',
                                                 target_size = (255, 255),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')


Found 10031 images belonging to 7 classes.
Found 193 images belonging to 7 classes.


In [ ]:
# dataVis = ImageList.from_folder('/content/drive/My Drive/TEES/Trabalho/dataset/')
# dataVis.open(dataVis.items[randint(0,len(dataVis))])

NameError: ignored

In [ ]:
# import warnings
# warnings.filterwarnings('ignore')

In [ ]:
# labels = ['akiec', 'bcc', 'bkl', 'df', 'mel', 'nv', 'vasc']
# for i in range (0, len(metadata.dx)):
#   if metadata.dx[i] == 'akiec':
#     metadata.dx[i] = 0
#   elif metadata.dx[i] == 'bcc':
#     metadata.dx[i] = 1
#   elif metadata.dx[i] == 'bkl':
#     metadata.dx[i] = 2
#   elif metadata.dx[i] == 'df':
#     metadata.dx[i] = 3
#   elif metadata.dx[i] == 'mel':
#     metadata.dx[i] = 4
#   elif metadata.dx[i] == 'nv':
#     metadata.dx[i] = 5
#   elif metadata.dx[i] == 'vasc':
#     metadata.dx[i] = 6


In [ ]:
# # y = metadata.dx.to_numpy(dtype = int)
# X_train, X_test, y_train, y_test = train_test_split(data, test_size = 0.3, random_state = 0)

# print('Total de imagens no dataset de treino:', len(X_train))
# print('Total de imagens no dataset de teste:', len(X_test))

ValueError: ignored

In [ ]:
# y = tf.convert_to_tensor(y)
# type(y)

<class 'int'>


tensorflow.python.framework.ops.EagerTensor

In [ ]:
# for i in range (0, len(X_train)):
#   if i < len(X_test):
#     X_train[i] = np.array(X_train)
#     X_train[i] = tf.convert_to_tensor(X_train[i])
#     X_test[i] = np.array(X_train)
#     X_test[i] = tf.convert_to_tensor
#   else:
#     X_train[i] = np.array(X_train)
#     X_train[i] = tf.convert_to_tensor(X_train[i])

ValueError: ignored

In [6]:
cnn = tf.keras.Sequential()
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[255, 255, 3]))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

cnn.add(tf.keras.layers.Flatten())
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

cnn.add(tf.keras.layers.Dense(units=7, activation='sigmoid'))


In [8]:
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [9]:
cnn.fit(x = data, validation_data = data_val, epochs = 25)

Epoch 1/25
314/314 [==============================] - 2015s 6s/step - loss: 0.2414 - accuracy: 0.6669 - val_loss: 0.2186 - val_accuracy: 0.6373
Epoch 2/25
314/314 [==============================] - 266s 848ms/step - loss: 0.2076 - accuracy: 0.6781 - val_loss: 0.1968 - val_accuracy: 0.7098
Epoch 3/25
314/314 [==============================] - 267s 849ms/step - loss: 0.1982 - accuracy: 0.6958 - val_loss: 0.1903 - val_accuracy: 0.6788
Epoch 4/25
314/314 [==============================] - 266s 847ms/step - loss: 0.1880 - accuracy: 0.7112 - val_loss: 0.1741 - val_accuracy: 0.7617
Epoch 5/25
314/314 [==============================] - 266s 847ms/step - loss: 0.1815 - accuracy: 0.7199 - val_loss: 0.1531 - val_accuracy: 0.7461
Epoch 6/25
314/314 [==============================] - 265s 843ms/step - loss: 0.1794 - accuracy: 0.7248 - val_loss: 0.1576 - val_accuracy: 0.7254
Epoch 7/25
314/314 [==============================] - 267s 850ms/step - loss: 0.1733 - accuracy: 0.7339 - val_loss: 0.1634 - v